In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("ProjectTweets.csv")

In [4]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [5]:
# Create a new DataFrame with the first row containing the original column names
new_row = pd.DataFrame([df.columns], columns=df.columns)

# Concatenate the new row with the original DataFrame
df = pd.concat([new_row, df], ignore_index=True)

df.reset_index(drop=True, inplace=True)

In [6]:
df.columns = ["n/a", "tweet_id", "date", "flag", "user", "text"]
df.head()

,n/a,tweet_id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   n/a       1600000 non-null  object
 1   tweet_id  1600000 non-null  object
 2   date      1600000 non-null  object
 3   flag      1600000 non-null  object
 4   user      1600000 non-null  object
 5   text      1600000 non-null  object
dtypes: object(6)
memory usage: 73.2+ MB


#### Loading data into MongoDB (NoSql)

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import to_timestamp

In [13]:
spark.stop()

In [8]:
#spark = SparkSession.builder.appName('twitter_analysis').getOrCreate()

In [14]:
# run this!
spark = SparkSession.builder.appName("MongoDB") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12-10.2.0") \
    .config("spark.mongodb.connection.uri", "mongodb://localhost:27017/") \
    .config("spark.mongodb.database", "twitter_db") \
    .config("spark.mongodb.collection", "tweets") \
    .getOrCreate()

In [8]:
spark = SparkSession.builder.appName("MongoDB") \
    .config("spark.jars", "/usr/local/spark/jars/mongo-spark-connector_2.12-10.2.0.jar") \
    .config("spark.mongodb.connection.uri", "mongodb://localhost:27017/") \
    .config("spark.mongodb.database", "twitter_db") \
    .config("spark.mongodb.collection", "tweets") \
    .getOrCreate()

In [15]:
print(spark.sparkContext.getConf().toDebugString())

spark.app.id=local-1697729444613
spark.app.name=MongoDB
spark.app.startTime=1697729444351
spark.driver.host=10.0.2.15
spark.driver.port=39299
spark.executor.id=driver
spark.jars.packages=org.mongodb.spark:mongo-spark-connector_2.12-10.2.0
spark.master=local[*]
spark.mongodb.client.factory=com.mongodb.spark.sql.connector.DefaultSource
spark.mongodb.collection=tweets
spark.mongodb.connection.uri=mongodb://localhost:27017/
spark.mongodb.database=twitter_db
spark.rdd.compress=True
spark.serializer.objectStreamReset=100
spark.sql.catalogImplementation=hive
spark.sql.warehouse.dir=file:/home/hduser/Desktop/CA/spark-warehouse
spark.submit.deployMode=client
spark.submit.pyFiles=
spark.ui.showConsoleProgress=true


In [16]:
my_schema = StructType([ StructField("n/a", StringType(), True)\
                   ,StructField("tweet_id", StringType(), True)\
                      ,StructField("date", StringType(), True)\
                      ,StructField("flag", StringType(), True)\
                      ,StructField("user", StringType(), True)\
                      ,StructField("text", StringType(), True)])

In [17]:
data = spark.createDataFrame(df, schema=my_schema)

In [13]:
data.show(5)

2023-10-19 15:27:24,448 WARN scheduler.TaskSetManager: Stage 0 contains a task of very large size (194599 KiB). The maximum recommended task size is 1000 KiB.


+---+----------+--------------------+--------+---------------+--------------------+
|n/a|  tweet_id|                date|    flag|           user|                text|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
+---+----------+--------------------+--------+---------------+--------------------+
only showing top 5 rows



In [26]:
# exploring rdd and pymongo

In [13]:
rdd = data.rdd

In [14]:
!pip install pymongo

Defaulting to user installation because normal site-packages is not writeable


In [16]:
from pymongo import MongoClient

def saveToMongoDB(iter):
    client = MongoClient("mongodb://127.0.0.1:27017")
    db = client["twitter_db"]
    collection = db["tweets"]
    for item in iter:
        # Your logic to convert RDD items to MongoDB documents
        # In this example, we assume the RDD contains key-value pairs
        document = {"key": item[0], "value": item[1]}
        collection.insert_one(document)
    client.close()

In [17]:
rdd.foreachPartition(saveToMongoDB)

2023-10-19 14:31:27,300 WARN scheduler.TaskSetManager: Stage 1 contains a task of very large size (194599 KiB). The maximum recommended task size is 1000 KiB.
2023-10-19 14:31:30,012 ERROR executor.Executor: Exception in task 0.0 in stage 1.0 (TID 1)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2932, in pipeline_func
    self.is_barrier = isFromBarrier or prev._is_barrier()
  File "/usr/local/spark/python/pyspark/rdd.py", line 2932, in pipeline_func
    self.is_barrier = isFromBarrier or prev._is_barrier()
  File "/usr/local/spark/python/pyspark/rdd.py", line 2932, in pipeline_func
    self.is_barrier = isFromBarrier or prev._is_barrier()
  File "/usr/local/spark/python/

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (10.0.2.15 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2932, in pipeline_func
    self.is_barrier = isFromBarrier or prev._is_barrier()
  File "/usr/local/spark/python/pyspark/rdd.py", line 2932, in pipeline_func
    self.is_barrier = isFromBarrier or prev._is_barrier()
  File "/usr/local/spark/python/pyspark/rdd.py", line 2932, in pipeline_func
    self.is_barrier = isFromBarrier or prev._is

In [20]:
mongo_config = {
    "mongo.input.uri": "mongodb://localhost:27017/twitter_db.tweets",
    "mongo.output.uri": "mongodb://localhost:27017/twitter_db.tweets",
    "mongo.output.format": "com.mongodb.hadoop.MongoOutputFormat"
}

In [23]:
# Save the RDD to MongoDB using Hadoop connectors
rdd.saveAsNewAPIHadoopFile(
    path="unused",
    outputFormatClass="com.mongodb.hadoop.MongoOutputFormat",
    keyClass="org.apache.hadoop.io.Text",
    keyConverter="org.apache.spark.examples.pythonconverters.WritableConverter",
    valueConverter="org.apache.spark.examples.pythonconverters.WritableConverter",
    conf=mongo_config
)

2023-10-19 14:44:12,405 WARN scheduler.TaskSetManager: Stage 2 contains a task of very large size (194599 KiB). The maximum recommended task size is 1000 KiB.
Traceback (most recent call last):                                  (0 + 1) / 1]
2023-10-19 14:44:13,453 ERROR executor.Executor: Exception in task 0.0 in stage 2.0 (TID 2)
net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.sql.types._create_row)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:213)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2(SerDeUtil.scala:121)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.saveAsNewAPIHadoopFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (10.0.2.15 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.sql.types._create_row)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:213)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2(SerDeUtil.scala:121)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at

In [24]:
# dataframe more suited

In [19]:
data.write.format("mongodb").mode("overwrite").save()

Py4JJavaError: An error occurred while calling o234.save.
: com.mongodb.spark.sql.connector.exceptions.ConfigException: Invalid value com.mongodb.spark.sql.connector.connection.DefaultMongoClientFactory for configuration mongoClientFactory
	at com.mongodb.spark.sql.connector.config.ClassHelper.createInstance(ClassHelper.java:79)
	at com.mongodb.spark.sql.connector.config.ClassHelper.createInstance(ClassHelper.java:39)
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.getMongoClientFactory(AbstractMongoConfig.java:232)
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.getMongoClient(AbstractMongoConfig.java:127)
	at com.mongodb.spark.sql.connector.config.WriteConfig.getMongoClient(WriteConfig.java:38)
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.withCollection(AbstractMongoConfig.java:163)
	at com.mongodb.spark.sql.connector.config.WriteConfig.withCollection(WriteConfig.java:38)
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.do

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "/usr/lib/python3/dist-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1871, in _run_once
    event_list = self._selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 469, in select
    fd

#### Data Processing with MapReduce/Spark

In [13]:
# TODO

#### Storing Post Processed Data

In [14]:
# TODO